In [ ]:
#016
sum([int(x) for x in str(2**1000)])

In [ ]:
#015
def fact(x):
    if x<=1:
        return x
    else:
        return fact(x-1)*x
print(fact(40)//fact(20)**2)

In [ ]:
#071
import fractions
N = 1000000
m = 1
num = set()
v0 = fractions.Fraction(3,7)
for i in range(N,0,-1):
    j = i*3//7
    v1 = fractions.Fraction(j,i)
    if v1 != v0:
        if v0 - v1 < m:
            m = v0-v1
            mv = v1
print(mv)


In [ ]:
#077

def primes(x):
    sieve = [True for i in range(x)]
    for i in range(2,x):
        if sieve[i]:
            for j in range(2*i,x,i):
                sieve[j] = False
    p = []
    for i in range(2,x):
        if sieve[i]:
            p.append(i)
    return p

numsumcache = dict()


def numsum(x,maxp):
    if (x,maxp) in numsumcache:
        return numsumcache[(x,maxp)]
    count = 0
    for i in p:
        if i <= maxp:
            if i<x:
                count += numsum(x-i,min(i,maxp))
            elif i==x:
                #print(x,maxp,seq+[i])
                count += 1
    numsumcache[(x,maxp)] = count
    return count

p = primes(5000)
i = 2
while numsum(i,i) < 5100:
    print(i,numsum(i,i))
    i+=1

In [ ]:
#056
m = 0
for a in range(100):
    for b in range(100):
        d = [int(c) for c in str(a**b)]
        e = sum(d)
        if m < e:
            m=e
            ma = a
            mb = b
print(ma,mb,m)

In [ ]:
#095
#divisors
import itertools
import time

divcache = dict()
divcache[1] = set([1])
p = set()

def divisors(x):
    if x in divcache:
        return divcache[x]
    sqx = int(x**0.5)+1
    for pn in range(1,sqx):
        if x % pn == 0:
            s = set()
            for j in divisors(x//pn):
                for k in divisors(pn):
                    s.add(j*k)
            divcache[x] = s
            return s
    #print("{0} is a prime number.".format(x),p)
    p.add(x)
    divcache[x] = set([1,x])
    return (1,x)

#100001:                             
now = time.time()
for i in range(1,100001):
    divisors(i)
print("time: {0}".format(time.time() - now))

RuntimeError: maximum recursion depth exceeded in comparison